In [13]:
import pickle
import numpy as np
import pandas as pd

In [14]:
DAY = "day_1"

In [15]:
with open(f"data/{DAY}/raw.pkl", "rb") as f:
    sensor_data = pickle.load(f)

In [16]:
def build_comparison_df(sensor_data, mat, sensor):
    data_lengths = []
    for heater_idx in sensor_data[mat][sensor]:
        heater_data_len = sensor_data[mat][sensor][heater_idx].shape[0]
        data_lengths.append(heater_data_len)

    min_len = min(data_lengths)
    heater_dict = {}
    for heater_idx in sensor_data[mat][sensor]:
        heater_dates = sensor_data[mat][sensor][heater_idx]["Date"].to_numpy()
        heater_data_len = len(heater_dates)
        if heater_data_len > min_len:
            heater_dates = heater_dates[:min_len]
        heater_dict[heater_idx] = heater_dates

    for heater_idx in range(1, 10):
        diff_seconds = (heater_dict[0] - heater_dict[heater_idx]
                        ).astype("timedelta64[s]").astype(np.int32)
        heater_dict[f"d 0-{heater_idx}"] = diff_seconds

    df = pd.DataFrame(heater_dict)
    return df

In [17]:
def impute_data_mean(df, after):
    df_copy = df.copy()
    line = df_copy.iloc[after:after+2].mean()

    line["Sensor Index"] = int(line["Sensor Index"])
    line["Sensor ID"] = int(line["Sensor ID"])
    line["Time Since PowerOn"] = int(line["Time Since PowerOn"])
    line["Real time clock"] = int(line["Real time clock"])
    line["Heater Profile Step Index"] = int(line["Heater Profile Step Index"])
    line["Scanning Mode Enabled"] = int(line["Scanning Mode Enabled"])
    line["Scanning Cycle Index"] = int(line["Scanning Cycle Index"])
    line["Label Tag"] = int(line["Label Tag"])
    line["Error Code"] = int(line["Error Code"])
    line["Date"] = line["Date"].floor("S")

    index = df_copy.iloc[after:after+2].index
    index = int((index[0] + index[1]) / 2)
    df_copy.loc[index] = line
    df_copy.sort_index(inplace=True)
    return df_copy

In [12]:
for matrix in ["air", "sample"]:
    for i in range(0, 8):
        df = build_comparison_df(sensor_data, matrix, i)
        df.to_excel(f"mat_{matrix}_s{i}_{DAY}_diff.xlsx")

In [23]:
sensor_data["air"][0][8].iloc[103:106]

,Sensor Index,Sensor ID,Time Since PowerOn,Real time clock,Temperature,Pressure,Relative Humidity,Resistance Gassensor,Heater Profile Step Index,Scanning Mode Enabled,Scanning Cycle Index,Label Tag,Error Code,Date
10575,0,355892538,2868013,1752327334,27.561605,922.078247,27.096571,4595792.5,8,1,1,0,0,2025-07-12 16:35:34
10678,0,355892538,2895412,1752327362,27.591656,922.061096,27.174850,4584219.5,8,1,1,0,0,2025-07-12 16:36:02
10877,0,355892538,2950076,1752327417,27.676798,922.085083,27.398743,4561247.0,8,1,1,0,0,2025-07-12 16:36:57


In [24]:
sensor = 0
heater = 8
after = 104
df_copy = impute_data_mean(sensor_data["air"][sensor][heater], after)
sensor_data["air"][sensor][heater] = df_copy.copy()
df_copy.iloc[after-1:after+3]

,Sensor Index,Sensor ID,Time Since PowerOn,Real time clock,Temperature,Pressure,Relative Humidity,Resistance Gassensor,Heater Profile Step Index,Scanning Mode Enabled,Scanning Cycle Index,Label Tag,Error Code,Date
10575,0,355892538,2868013,1752327334,27.561605,922.078247,27.096571,4595792.50,8,1,1,0,0,2025-07-12 16:35:34
10678,0,355892538,2895412,1752327362,27.591656,922.061096,27.174850,4584219.50,8,1,1,0,0,2025-07-12 16:36:02
10777,0,355892538,2922744,1752327389,27.634227,922.073090,27.286797,4572733.25,8,1,1,0,0,2025-07-12 16:36:29
10877,0,355892538,2950076,1752327417,27.676798,922.085083,27.398743,4561247.00,8,1,1,0,0,2025-07-12 16:36:57


In [ ]:
# fix broken readings
# sensor_data[MATRIX][1] = sensor_data[MATRIX][0].copy()

In [7]:
with open(f"data/{DAY}/imputed.pkl", "wb") as f:
    pickle.dump(sensor_data, f)